In [2]:
import scanpy as sc
import pandas as pd

from clustering_utils import decompose

sc.set_figure_params(frameon=False, dpi=150, figsize=(8, 8))

In [3]:
adata = sc.read_h5ad('../data/neuronal_raw_embed_propagated.h5ad')
adata

AnnData object with n_obs × n_vars = 522212 × 56939
    obs: 'Author', 'Author_CellType', 'Author_Class_Curated', 'C185_named', 'C25_named', 'C286_named', 'C2_named', 'C465_named', 'C66_named', 'C7_named', 'Cell_ID', 'Dataset', 'Macosko_CellType', 'Region', 'Region_predicted', 'Sample', 'Sample_ID', 'Source', 'Suspension', 'Suspension_bin', 'Technology', 'class_id_label', 'cluster_id_label', 'core', 'is_control', 'nt_type_label', 'pct_mito', 'subclass_id_label', 'supertype_id_label', 'total_counts', 'V1_class_propagated', 'V1_class_confidence', 'ABC_class_propagated', 'ABC_class_confidence', 'major_features', 'major_features_propagated', 'major_features_confidence'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'ABC_class_propagated_colors', 'Author_Class_Curated_colors', 'Author_colors', 'Suspension_bin_mapping', 'V1_class_propagated_colors', 'major_features_colors', 'major_features_propagated_colors', 'neighbors', 'tsne', 'umap'
    obsm: 'X_embed', 'X_tsne', 'X_umap'
    ob

In [4]:
mrtree_labelmat = pd.read_csv('../data/neuronal_mrtree_labelmat.csv', index_col=0)
mrtree_labelmat.head()

,ABC_class_propagated,r1_consensus,r8_consensus,r40_consensus,r80_consensus
AAACCCAAGATGCTTC_v2-19-A006200239-175241,3,16,179,46,323
AAACCCAGTACCTAAC_v2-19-A006200239-175241,0,34,90,74,485
AAACCCATCCGAAATC_v2-19-A006200239-175241,3,15,19,207,996
AAACCCATCGTAGAGG_v2-19-A006200239-175241,0,10,124,652,397
AAACCCATCTTAGTTC_v2-19-A006200239-175241,3,49,146,288,303


In [5]:
adata.obs['temp'] = mrtree_labelmat.iloc[:, -2]

In [6]:
mrtree_labelmat.iloc[:, -1] = decompose(adata, 'temp', 1, 100)

Decomposing Clusters: 100%|██████████| 749/749 [09:36<00:00,  1.30it/s]


In [8]:
mrtree_labelmat.iloc[:, -1].nunique()

6594

In [ ]:
df = adata.obs['temp'].str.split('_', expand=True)
df.columns = ['prefix', 'suffix']

# Convert prefix to integer if needed (for correct sorting/grouping)
df['prefix'] = df['prefix'].astype(str)
df['suffix'] = df['suffix'].astype(str)

# Group by prefix and count the number of unique suffixes
cluster_divisions = df.groupby('prefix')['suffix'].nunique()

cluster_divisions.hist(bins=30)

In [10]:
mrtree_labelmat.to_csv('../data/neuronal_raw_labelmat.csv')